In [6]:
from pathlib import Path
import struct
import bitstruct
import cbitstruct
from pcapng import FileScanner
from pcapng.blocks import EnhancedPacket

from time import perf_counter
import numpy as np

In [7]:
PACKET_HEADER_LENGTH = 42
SOURCE_OFFSET = 26
SOURCE_LENGTH = 4

ETHER_HEADER_LENGTH = 4
ETHER_BODY_OFFSET = 2
EVENT_STRIDE_BYTES = 12

CHECK_XY_RANGE = range(85, 95)
CHECK_QT_RANGE = range(125, 135)

In [20]:
def translate_msg_to_photon_list(
    msg: bytes,
) -> tuple[int, list[tuple[int, int, int]], int]:
    header_bytes, msg = (
        msg[:ETHER_HEADER_LENGTH],
        msg[ETHER_HEADER_LENGTH + ETHER_BODY_OFFSET :],
    )
    n_events, packet_id = struct.unpack("<HH", header_bytes)
    n_events //= 2

    events = []
    n_failed = 0

    for i in range(n_events):
        xy_data = msg[
            EVENT_STRIDE_BYTES * i : EVENT_STRIDE_BYTES * i + EVENT_STRIDE_BYTES // 2
        ]
        qt_data = msg[
            EVENT_STRIDE_BYTES * i
            + EVENT_STRIDE_BYTES // 2 : EVENT_STRIDE_BYTES * (i + 1)
        ]
        x, y, check_xy = bitstruct.unpack("u16u16u16<", xy_data)
        q, t, check_qt = bitstruct.unpack("u16u16u16<", qt_data)
        # Sometimes xy and qt are swapped so we can check static bytes to see if they need to be swapped back

        if check_qt in CHECK_QT_RANGE:
            events.append((x, y, t))
        else:
            events.append((q, t, y))

        # It seems like the values have changed after a long downtime
        # TODO: figure out a robust way to do this check
        # if (check_xy in CHECK_XY_RANGE) and (check_qt in CHECK_QT_RANGE):
        #     events.append((x, y, t))
        # elif (check_xy in CHECK_QT_RANGE) and (check_qt in CHECK_XY_RANGE):
        #     events.append((q, t, y))
        # else:
        #     n_failed += 1

    return packet_id, events, n_failed


def fast_translate_0(
    msg: bytes,
) -> tuple[int, list[tuple[int, int, int]], int]:
    header_bytes, msg = (
        msg[:ETHER_HEADER_LENGTH],
        msg[ETHER_HEADER_LENGTH + ETHER_BODY_OFFSET :],
    )
    n_events, packet_id = struct.unpack("<HH", header_bytes)
    n_events //= 2

    events = []
    n_failed = 0

    for i in range(n_events):
        xy_data = msg[
            EVENT_STRIDE_BYTES * i : EVENT_STRIDE_BYTES * i + EVENT_STRIDE_BYTES // 2
        ]
        qt_data = msg[
            EVENT_STRIDE_BYTES * i
            + EVENT_STRIDE_BYTES // 2 : EVENT_STRIDE_BYTES * (i + 1)
        ]
        x, y, check_xy = cbitstruct.unpack("u16u16u16<", xy_data)
        q, t, check_qt = cbitstruct.unpack("u16u16u16<", qt_data)
        # Sometimes xy and qt are swapped so we can check static bytes to see if they need to be swapped back

        if check_qt in CHECK_QT_RANGE:
            events.append((x, y, t))
        else:
            events.append((q, t, y))

        # It seems like the values have changed after a long downtime
        # TODO: figure out a robust way to do this check
        # if (check_xy in CHECK_XY_RANGE) and (check_qt in CHECK_QT_RANGE):
        #     events.append((x, y, t))
        # elif (check_xy in CHECK_QT_RANGE) and (check_qt in CHECK_XY_RANGE):
        #     events.append((q, t, y))
        # else:
        #     n_failed += 1

    return packet_id, events, n_failed


def fast_translate_1(msg: bytes) -> tuple[int, list[tuple[int, int, int]], int]:
    header_bytes, msg = (
        msg[:ETHER_HEADER_LENGTH],
        msg[ETHER_HEADER_LENGTH + ETHER_BODY_OFFSET :],
    )
    n_events, packet_id = struct.unpack("<HH", header_bytes)
    n_events //= 2

    n_failed = 0

    event_data = msg[0 : EVENT_STRIDE_BYTES * n_events]
    all_data = np.array(cbitstruct.unpack(f"{'u16'*6*n_events}<", event_data))
    all_data = all_data.reshape(n_events, 6)
    if all_data[0, 3] == 0:
        return packet_id, all_data[:, [0, 1, 4]], n_failed
    else:
        return packet_id, all_data[:, [1, 3, 4]], n_failed


def fast_translate_2(msg: bytes) -> tuple[int, list[tuple[int, int, int]], int]:
    header_bytes, msg = (
        msg[:ETHER_HEADER_LENGTH],
        msg[ETHER_HEADER_LENGTH + ETHER_BODY_OFFSET :],
    )
    n_events, packet_id = struct.unpack("<HH", header_bytes)
    n_events //= 2

    n_failed = 0

    event_data = msg[0 : EVENT_STRIDE_BYTES * n_events]
    all_data = np.array(cbitstruct.unpack(f"{'u16'*6*n_events}<", event_data))
    all_data = all_data.reshape(n_events, 6)
    if np.mean(all_data[:, 3]) == 0:
        return packet_id, all_data[:, [0, 1, 4]], n_failed
    else:
        return packet_id, all_data[:, [3, 4, 1]], n_failed


def decode_block(block):
    if not isinstance(block, EnhancedPacket):
        return

    header = block.packet_data[:PACKET_HEADER_LENGTH]
    source_ip = bitstruct.unpack(
        f"{'u8'*4}<", header[SOURCE_OFFSET : SOURCE_OFFSET + SOURCE_LENGTH]
    )
    if source_ip != (192, 168, 0, 5):
        return

    return block.packet_data[PACKET_HEADER_LENGTH:]

In [21]:
packet_path = r"C:\Users\SPEEM\Documents\speem\speem\scripts\correct packets.pcapng"

with open(packet_path, "rb") as f:
    file_scanner = FileScanner(f)
    slow_times = []
    fast0_times = []
    fast1_times = []
    fast2_times = []
    for block in file_scanner:
        msg = decode_block(block)
        if msg is not None:
            start_slow = perf_counter()
            slow_id, slow_events, slow_fails = translate_msg_to_photon_list(msg)
            end_slow = perf_counter()
            fast0_id, fast0_events, fast0_fails = fast_translate_0(msg)
            end_fast_0 = perf_counter()
            fast1_id, fast1_events, fast1_fails = fast_translate_1(msg)
            end_fast_1 = perf_counter()
            fast2_id, fast2_events, fast2_fails = fast_translate_2(msg)
            end_fast_2 = perf_counter()

            slow_times.append(end_slow - start_slow)
            fast0_times.append(end_fast_0 - end_slow)
            fast1_times.append(end_fast_1 - end_fast_0)
            fast2_times.append(end_fast_2 - end_fast_1)

            assert slow_id == fast0_id == fast1_id == fast2_id

slow_times = np.array(slow_times)
fast0_times = np.array(fast0_times)
fast1_times = np.array(fast1_times)
fast2_times = np.array(fast2_times)

print(f"Slow mean: {slow_times.mean():.2e}, std: {slow_times.std():.2e}")
print(f"Fast_0 mean: {fast0_times.mean():.2e}, std: {fast0_times.std():.2e}")
print(f"Fast_1 mean: {fast1_times.mean():.2e}, std: {fast1_times.std():.2e}")
print(f"Fast_2 mean: {fast2_times.mean():.2e}, std: {fast2_times.std():.2e}")

Slow mean: 6.11e-03, std: 2.54e-04
Fast_0 mean: 1.83e-04, std: 1.98e-05
Fast_1 mean: 5.34e-05, std: 1.06e-05
Fast_2 mean: 6.67e-05, std: 1.26e-05


In [10]:
for slow_event, fast_event_0, fast_event_1 in zip(
    slow_events, fast0_events, fast1_events
):
    assert list(slow_event) == list(fast_event_0) == list(fast_event_1)

In [18]:
def translate_packet(msg: bytes) -> tuple[int, list[tuple[int, int, int]], int]:
    header_bytes, msg = (
        msg[:ETHER_HEADER_LENGTH],
        msg[ETHER_HEADER_LENGTH + ETHER_BODY_OFFSET :],
    )
    n_events, packet_id = struct.unpack("<HH", header_bytes)
    n_events //= 2

    event_data = msg[0 : EVENT_STRIDE_BYTES * n_events]
    all_data = np.array(cbitstruct.unpack(f"{'u16'*6*n_events}<", event_data))
    all_data = all_data.reshape(n_events, 6)
    return all_data

    if all_data[0, 3] == 0:
        return packet_id, all_data[:, [0, 1, 4]], n_failed
    else:
        return packet_id, all_data[:, [3, 4, 1]], n_failed


root = Path(r"C:\Users\SPEEM\Documents\speem\speem\scripts")
correct_packets = root / "correct_order packets.pcapng"
reversed_packets = root / "reversed packets.pcapng"

for packet_file in [correct_packets, reversed_packets]:
    average_3rd = []
    with open(packet_file, "rb") as f:
        file_scanner = FileScanner(f)
        for block in file_scanner:
            msg = decode_block(block)
            if msg is not None:
                packet_data = translate_packet(msg)
                break
                average_3rd.append(np.mean(packet_data[:, 3]))
    # print(np.mean(average_3rd))

[[2321 2712    9    0 4095  129]
 [1692 2047    7    0 4095  129]
 [1563 2612    9    0 4095  129]
 [1422 2396    1    0 4095  129]
 [2138 2552    4    0 4095  129]
 [1818 2723    1    0 4095  129]
 [1479 2275   14    0 4095  129]
 [1505 2376    8    0 4095  129]
 [1602 2062    4    0 4095  129]
 [1994 2223    5    0 4095  129]
 [2393 2295    8    0 4095  129]
 [1304 2040    4    0 4095  129]
 [1536 2628    2    0 4095  129]
 [1114 2480    5    0 4095  129]
 [2017 2748    5    0 4095  129]
 [1746 2342   12    0 4095  129]
 [2676 2629    8    0 4095  129]
 [2038 2539    8    0 4095  129]
 [1648 2300    6    0 4095  129]
 [1483 2191    5    0 4095  129]
 [1485 2553    5    0 4095  129]
 [1376 2401    7    0 4095  129]
 [1986 2366    9    0 4095  129]
 [2673 1837   12    0 4095  129]
 [1383 2099    6    0 4095  129]
 [1339 2451    8    0 4095  129]
 [1731 2539   10    0 4095  129]
 [1266 2535    3    0 4095  129]
 [2295 2692   10    0 4095  129]
 [2054 2495    9    0 4095  129]
 [2534 251